In [13]:
import cv2 as cv
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn
import os
import shutil
import pywt
from sklearn.preprocessing import StandardScaler
from sklearn.svm import SVC
from sklearn.model_selection import train_test_split
from sklearn.pipeline import Pipeline
from sklearn import svm
from sklearn.ensemble import RandomForestClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.pipeline import make_pipeline
from sklearn.model_selection import GridSearchCV
from tensorflow import keras
import tensorflow as tf


In [14]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [15]:
eye_cascade = cv.CascadeClassifier('/content/drive/MyDrive/Colab Notebooks/Sports Person Detector/model/opencv/haarcascade_eye.xml')
face_cascade = cv.CascadeClassifier('/content/drive/MyDrive/Colab Notebooks/Sports Person Detector/model/opencv/haarcascade_frontalface_default.xml')

In [16]:
# making a function to input an image and getthe cropped photo only if there are two eyes visible
def get_cropped_img(image_path):
    img = cv.imread(image_path)
    if ['i'] is not None:
        gray=cv.cvtColor(img, cv.COLOR_BGR2GRAY)
        faces = face_cascade.detectMultiScale(gray,1.3,5)
        for x,y,w,h in faces:
            roi_gray = gray[y:y+h,x:x+w]
            roi_color = img[y:y+h,x:x+w]
            eyes = eye_cascade.detectMultiScale(roi_gray)
            if len(eyes)>=2:
                return roi_color


In [ ]:
# cropped1 = get_cropped_img('test_images/test1.jpg')
# cropped2 = get_cropped_img('test_images/test2.jpeg')
# print('FACE') if cropped2 else print('NO FACE')/
# we can see that the image test2 does not have a clear face so we wont use images like that to train the model

In [22]:
path_to_data = '/content/drive/MyDrive/Colab Notebooks/Sports Person Detector/model/dataset'
path_to_cr_data = '/content/drive/MyDrive/Colab Notebooks/Sports Person Detector/model/dataset/cropped/'

In [23]:
img_dirs = []
for entry in os.scandir(path_to_data):
    if entry.is_dir():
        img_dirs.append(entry.path)

In [24]:
# checking whether a cropped file is created or not
if os.path.exists(path_to_cr_data):
    shutil.rmtree(path_to_cr_data)
os.mkdir(path_to_cr_data)

In [28]:
cropped_image_dirs=[]
celebrity_file_name_dict={}

for img_dir in img_dirs:
    count=1
    celebrity_name = img_dir.split('/')[-1]
    print(celebrity_name)

    celebrity_file_name_dict[celebrity_name]=[]
    for entry in os.scandir(img_dir):
      if img_dir == '/content/drive/MyDrive/Colab Notebooks/Sports Person Detector/model/dataset/cropped':
        break
      # print(img_dir)
      roi_color = get_cropped_img(entry.path)
      if roi_color is not None:
          cropped_folder = path_to_cr_data+celebrity_name
          if not os.path.exists(cropped_folder):
              os.makedirs(cropped_folder)
              cropped_image_dirs.append(cropped_folder)
              print('Generating Cropped Image in - ',cropped_folder)



          cropped_file_name = celebrity_name+str(count)+'.png'
          cropped_file_path = cropped_folder+'/'+cropped_file_name

          cv.imwrite(cropped_file_path,roi_color)
          celebrity_file_name_dict[celebrity_name].append(cropped_file_path)
          count+=1

scarlett_johansson
hugh_jackman
robert_downey_jr
brad_pitt
angelina_jolie
cropped


In [29]:

celeb_dict = {}
count = 0
for celebrity_name in celebrity_file_name_dict.keys():
    celeb_dict[celebrity_name] = count
    count = count + 1
celeb_dict.popitem()
celeb_dict

{'scarlett_johansson': 0,
 'hugh_jackman': 1,
 'robert_downey_jr': 2,
 'brad_pitt': 3,
 'angelina_jolie': 4}

In [30]:
def w2d(img, mode='haar', level=1):
    imArray = img
    #Datatype conversions
    #convert to grayscale
    imArray = cv.cvtColor( imArray,cv.COLOR_RGB2GRAY )
    #convert to float
    imArray =  np.float32(imArray)
    imArray /= 255;
    # compute coefficients
    coeffs=pywt.wavedec2(imArray, mode, level=level)

    #Process Coefficients
    coeffs_H=list(coeffs)
    coeffs_H[0] *= 0;

    # reconstruction
    imArray_H=pywt.waverec2(coeffs_H, mode);
    imArray_H *= 255;
    imArray_H =  np.uint8(imArray_H)

    return imArray_H

In [31]:
X, y = [], []
for celebrity_name, training_files in celebrity_file_name_dict.items():
    for training_image in training_files:
        img = cv.imread(training_image)
        if img is None:
            continue
        gray = cv.cvtColor(img, cv.COLOR_BGR2GRAY)
        scaled_raw_img = cv.resize(gray, (32, 32))
        img_har = w2d(img,'db1',5)
        scaled_img_har = cv.resize(img_har, (32, 32))
        # combined_img = np.vstack((scaled_raw_img.reshape(32*32*3,1),scaled_img_har.reshape(32*32,1)))
        combined_img = np.vstack((scaled_raw_img , scaled_img_har))
        # print('scaled img' , scaled_raw_img.shape)
        # print('scaled har img' , scaled_img_har.shape)
        X.append(combined_img)
        y.append(celeb_dict[celebrity_name])

In [32]:
X = np.array(X).reshape(np.array(X).shape[0], 64, 32, 1)
X = X.astype(float)

In [33]:
X.shape

(463, 64, 32, 1)

In [34]:
X=X/255

In [35]:
y=np.asarray(y)
# y = y.reshape(549,-1)
xtrain,xtest,ytrain,ytest = train_test_split(X,y,test_size=0.1,random_state=546)

In [36]:
!pip install keras-tuner -q

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 128.0/128.0 kB 2.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 950.8/950.8 kB 33.3 MB/s eta 0:00:00


In [ ]:
# yt=[]
# for i in ytest:
#     if i==3:
#         yt.append(5)
#     elif i==5:
#         yt.append(3)
#     else:
#         yt.append(i)


In [37]:
model = keras.models.Sequential()
model.add(keras.layers.Conv2D(
    filters = 32,
    kernel_size = (3,3),
    strides = (1,1),
    padding='valid',
    activation='relu',
    input_shape=(64,32,1)))
model.add(keras.layers.Conv2D(
    filters =32,
    kernel_size = (3,3),
    strides = (1,1),
    padding='valid',
    activation='relu'))
model.add(keras.layers.MaxPool2D((2,2)))

model.add(keras.layers.Conv2D(
    filters = 32,
    kernel_size = (3,3),
    strides = (1,1),
    padding='valid',
    activation='relu'))
model.add(keras.layers.Conv2D(
    filters =32,
    kernel_size = (3,3),
    strides = (1,1),
    padding='valid',
    activation='relu'))
model.add(keras.layers.MaxPool2D((1,1)))

model.add(keras.layers.Flatten())
model.add(keras.layers.Dense(units=512,activation='relu'))
model.add(keras.layers.Dense(5,activation='softmax'))

# model.summary()
model.compile(loss='sparse_categorical_crossentropy',metrics=['accuracy'],optimizer=keras.optimizers.Adam(learning_rate=0.001))

In [38]:
def model_arch(filters,kernel,units,lr):
    model = keras.models.Sequential()

    # model.add(keras.layers.Conv2D(32, kernel_size=(5,5), strides = (1,1), padding='valid', activation='relu', input_shape=(64,32,1)))
    model.add(keras.layers.Conv2D(
        filters = filters,
        kernel_size = kernel,
        strides = (1,1),
        padding='valid',
        activation='relu',
        input_shape=(64,32,1)))
    model.add(keras.layers.Conv2D(
        filters =filters,
        kernel_size = kernel,
        strides = (1,1),
        padding='valid',
        activation='relu'))
    model.add(keras.layers.MaxPool2D((2,2)))

    model.add(keras.layers.Conv2D(
        filters = filters,
        kernel_size = kernel,
        strides = (1,1),
        padding='valid',
        activation='relu'))
    model.add(keras.layers.Conv2D(
        filters =filters,
        kernel_size = kernel,
        strides = (1,1),
        padding='valid',
        activation='relu'))
    model.add(keras.layers.MaxPool2D((1,1)))

    model.add(keras.layers.Flatten())
    model.add(keras.layers.Dense(units=units,activation='relu'))
    model.add(keras.layers.Dense(5,activation='softmax'))

    # model.summary()
    model.compile(loss='sparse_categorical_crossentropy',metrics=['accuracy'],optimizer=keras.optimizers.Adam(learning_rate=lr))
    return model

def build_model(hp):
    filters = hp.Int('conv_filter', min_value=32,max_value=64,step=16)
    kernel = hp.Choice('conv_kernel', values=[3,5])
    units = hp.Int('units' , min_value=128, max_value=512, step=32)
    # activation - hp.Choice('activation', values=['relu','tanh'])
    lr = hp.Choice('learning_rate', values=[0.001])
    model = model_arch(filters=filters,kernel=kernel,units=units,lr=lr)
    return model


In [39]:
import keras_tuner
build_model(keras_tuner.HyperParameters())

Using TensorFlow backend


In [40]:
tuner = keras_tuner.BayesianOptimization(
hypermodel = build_model,
objective = 'val_accuracy',
max_trials = 20,
overwrite=True,
directory = 'my_dir',
project_name='tuner_search',)

In [41]:
tuner.search(xtrain,ytrain,epochs=8,validation_data=(xtest,ytest))

Trial 20 Complete [00h 00m 07s]
val_accuracy: 0.7446808218955994

Best val_accuracy So Far: 0.8723404407501221
Total elapsed time: 00h 02m 27s


In [ ]:
tuner.results_summary()

Results summary
Results in my_dir/tuner_search
Showing 10 best trials
Objective(name="val_accuracy", direction="max")

Trial 13 summary
Hyperparameters:
conv_filter: 64
conv_kernel: 3
units: 480
learning_rate: 0.001
Score: 0.8297872543334961

Trial 15 summary
Hyperparameters:
conv_filter: 64
conv_kernel: 3
units: 480
learning_rate: 0.001
Score: 0.8297872543334961

Trial 19 summary
Hyperparameters:
conv_filter: 64
conv_kernel: 3
units: 480
learning_rate: 0.001
Score: 0.8297872543334961

Trial 16 summary
Hyperparameters:
conv_filter: 64
conv_kernel: 3
units: 480
learning_rate: 0.001
Score: 0.8085106611251831

Trial 17 summary
Hyperparameters:
conv_filter: 64
conv_kernel: 3
units: 480
learning_rate: 0.001
Score: 0.8085106611251831

Trial 02 summary
Hyperparameters:
conv_filter: 48
conv_kernel: 3
units: 224
learning_rate: 0.001
Score: 0.7872340679168701

Trial 18 summary
Hyperparameters:
conv_filter: 64
conv_kernel: 3
units: 480
learning_rate: 0.001
Score: 0.7872340679168701

Trial 07 summ

In [42]:
models = tuner.get_best_models(num_models=2)
best_model = models[0]
best_model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d (Conv2D)             (None, 62, 30, 64)        640       
                                                                 
 conv2d_1 (Conv2D)           (None, 60, 28, 64)        36928     
                                                                 
 max_pooling2d (MaxPooling2  (None, 30, 14, 64)        0         
 D)                                                              
                                                                 
 conv2d_2 (Conv2D)           (None, 28, 12, 64)        36928     
                                                                 
 conv2d_3 (Conv2D)           (None, 26, 10, 64)        36928     
                                                                 
 max_pooling2d_1 (MaxPoolin  (None, 26, 10, 64)        0         
 g2D)                                                   

In [44]:
model = keras.models.Sequential()
model.add(keras.layers.Conv2D(
    filters = 64,
    kernel_size = (3,3),
    strides = (1,1),
    padding='valid',
    activation='relu',
    input_shape=(64,32,1)))
model.add(keras.layers.Conv2D(
    filters =64,
    kernel_size = (3,3),
    strides = (1,1),
    padding='valid',
    activation='relu'))
model.add(keras.layers.MaxPool2D((2,2)))

model.add(keras.layers.Conv2D(
    filters =32,
    kernel_size = (3,3),
    strides = (1,1),
    padding='valid',
    activation='relu'))
model.add(keras.layers.Conv2D(
    filters =32,
    kernel_size = (3,3),
    strides = (1,1),
    padding='valid',
    activation='relu'))
model.add(keras.layers.MaxPool2D((1,1)))

model.add(keras.layers.Flatten())
model.add(keras.layers.Dense(units=512,activation='relu'))
model.add(keras.layers.Dense(5,activation='softmax'))

# model.summary()
model.compile(loss='sparse_categorical_crossentropy',metrics=['accuracy'],optimizer=keras.optimizers.Adam(learning_rate=0.001))
model.fit(xtrain,ytrain,epochs=7,validation_data=(xtest,ytest))

Epoch 1/7
13/13 [==============================] - 4s 40ms/step - loss: 1.5835 - accuracy: 0.3438 - val_loss: 1.3934 - val_accuracy: 0.4681
Epoch 2/7
13/13 [==============================] - 0s 16ms/step - loss: 1.4896 - accuracy: 0.3558 - val_loss: 1.3496 - val_accuracy: 0.4681
Epoch 3/7
13/13 [==============================] - 0s 24ms/step - loss: 1.1971 - accuracy: 0.5048 - val_loss: 1.3023 - val_accuracy: 0.4468
Epoch 4/7
13/13 [==============================] - 0s 15ms/step - loss: 0.9182 - accuracy: 0.6538 - val_loss: 1.3196 - val_accuracy: 0.5532
Epoch 5/7
13/13 [==============================] - 0s 19ms/step - loss: 0.6535 - accuracy: 0.7861 - val_loss: 0.7197 - val_accuracy: 0.7234
Epoch 6/7
13/13 [==============================] - 0s 17ms/step - loss: 0.4366 - accuracy: 0.8341 - val_loss: 1.1439 - val_accuracy: 0.5957
Epoch 7/7
13/13 [==============================] - 0s 17ms/step - loss: 0.3373 - accuracy: 0.8966 - val_loss: 0.5786 - val_accuracy: 0.8085


In [45]:
best_model.fit(xtrain,ytrain,epochs=5,validation_data=(xtest,ytest))


Epoch 1/5
13/13 [==============================] - 3s 38ms/step - loss: 0.1742 - accuracy: 0.9567 - val_loss: 0.5593 - val_accuracy: 0.7660
Epoch 2/5
13/13 [==============================] - 0s 16ms/step - loss: 0.0708 - accuracy: 0.9760 - val_loss: 0.4087 - val_accuracy: 0.8085
Epoch 3/5
13/13 [==============================] - 0s 16ms/step - loss: 0.0361 - accuracy: 0.9928 - val_loss: 0.5462 - val_accuracy: 0.8085
Epoch 4/5
13/13 [==============================] - 0s 13ms/step - loss: 0.0073 - accuracy: 1.0000 - val_loss: 0.6136 - val_accuracy: 0.8298
Epoch 5/5
13/13 [==============================] - 0s 16ms/step - loss: 0.0178 - accuracy: 0.9952 - val_loss: 0.6795 - val_accuracy: 0.7660
